setup

In [ ]:
import json, csv, glob, random, string, os
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType
from pyspark.sql.functions import col,isnan,when,count

In [ ]:
def read_csv(path):
    csvfile = open(path, newline='')
    reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    return list(reader)

def write_csv(path, data):
    f = open(path, 'w')
    writer = csv.writer(f)
    writer.writerow(data)
    f.close()

def read_json(path):
    return json.load(open(path))

def write_json(path, data):
    file = open(path, "w+")
    file.write(json.dumps(data)) 
    file.close()

def get_subpaths(path):
    return glob.glob(path)

def generate_cat(size):
    return ''.join(random.choice(string.ascii_lowercase + string.digits) for _ in range(size))

def write_tfrecord(dir, df):
    df.write.format("tfrecords").option("recordType", "Example").save(dir)

In [ ]:
ROOT_DIR = os.path.dirname(os.getcwd()) + "/"
ROOT_DIR

In [ ]:
os.environ['SPARK_HOME'] = ROOT_DIR + "resources/spark/"
os.environ['SPARK_HOME']
connector = ROOT_DIR + "resources/ecosystem/spark/spark-tensorflow-connector/target/spark-tensorflow-connector_2.12-1.11.0.jar"

In [ ]:
DATA_DIR = ROOT_DIR + "data/criteo/"


In [ ]:
TARGET_DIR = DATA_DIR + "train/"
# TARGET_DIR = DATA_DIR + "sample/"

In [ ]:
SCHEMA_PATH = DATA_DIR + "schema.json"
PART_DIRS = get_subpaths(TARGET_DIR + "part*/")
PART_DIRS

In [ ]:
spark = SparkSession.builder.appName("pCTR").\
    config('spark.jars', connector).\
    getOrCreate()

In [ ]:
schema = StructType.fromJson(json.load(open(SCHEMA_PATH)))

In [ ]:
num_features = ["i1", "i2", "i3", "i4", "i5", "i6", "i7", "i8", "i9", "i10", "i11", "i12", "i13"]

In [ ]:
def numeric(df, total, stats, num_features, normalizer = "minmax"):
    def bring_stat(stats, feature):
        keys = ["min", "max", "avg", "median", "mode", "nulls", "stddev"]
        stat = {}
        for key in keys:
            stat[key]= stats["{}({})".format(key, feature)]
        return stat 

    for feature in num_features:
        stat = bring_stat(stats, feature)
        # if mode is not 0, safe to assume missing = 0. (assuming non-negative data)
        if stat["mode"] != 0:
            df = df.fillna(value = 0, subset = [feature])
        else: # else if mode is 0, then:
            null_ratio = stat["nulls"] / total
            if null_ratio < 0.1: # if null ratio is low, safe to assume missing = 0
                df = df.fillna(value = 0, subset = [feature])
            elif null_ratio > 0.6: # if null percentage is high: use mean
                df = df.fillna(value = stat["avg"], subset = [feature])
            else:  # if null percentage is mid: use median
                df = df.fillna(value = stat["median"], subset = [feature])
        # normalizer
        if normalizer == "minmax": 
            df = df.withColumn(feature, (col(feature) - stat['min']) / (stat["max"] - stat["min"]))
        elif normalizer == "standard":
            df = df.withColumn(feature, (col(feature) - stat['avg']) / stat['stddev'])
    return df

In [ ]:
cat_features = ["c1", "c2", "c3", "c4", "c5", "c6", "c7", "c8", "c9", "c10", "c11", "c12", "c13", "c14", "c15", "c16", "c17", "c18", "c19", "c20", "c21", "c22", "c23", "c24", "c25", "c26"]

In [ ]:
def categorical(df, vocab_dir, cat_features):
    for feature in cat_features:
        json_path = vocab_dir + "{}/count.json".format(feature)
        null_feature = read_json(json_path)["null_feature"]
        # go through the column and mark null as the new feature.
        df = df.fillna(value = null_feature, subset = [feature])
    return df

In [ ]:
for part_dir in PART_DIRS:
    stats = read_json(part_dir + "stats/stats.json")
    total = read_json(part_dir + "count/count.json")["total"]

    data_path = get_subpaths(part_dir + "raw/*.txt")[0]
    df = spark.read.option("header", "false").option("delimiter", "\t").schema(schema).csv(data_path)

    df = numeric(df, total, stats, num_features, normalizer = "standard")
    df = categorical(df, part_dir + "vocabs/", cat_features)
    write_tfrecord(part_dir + "data/", df)

In [ ]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()